In [1]:
import xml.etree.ElementTree as ET
import json
import glob
import re
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open('../data/matching_process/filings.json', 'r') as json_file:
    data = json.load(json_file)

df = pd.read_csv('../data/findings/data-broker-filings.csv')

Create a key to match subsidiaries with their parent companies

In [3]:
subsidiaries_df = df[~df['parent_company'].isna()][['parent_company', 'company']].drop_duplicates().copy()
subsidiaries = pd.Series(subsidiaries_df.parent_company.values,index=subsidiaries_df.company).to_dict()

## Spending

In [4]:
period_spending = df.groupby(['company', 'period', 'is_self_filer']).sum().reset_index().copy()

In [5]:
period_spending = period_spending.sort_values(by='is_self_filer', ascending=False).drop_duplicates(subset=['company', 'period'])

In [6]:
period_spending

,company,period,is_self_filer,spending
87,ORACLE,1st Quarter (Jan 1 - Mar 31),True,1890000.0
103,PRICEWATERHOUSECOOPERS,3rd Quarter (July 1 - Sep 30),True,730000.0
105,PRICEWATERHOUSECOOPERS,4th Quarter (Oct 1 - Dec 31),True,690000.0
129,S&P GLOBAL,4th Quarter (Oct 1 - Dec 31),True,370000.0
50,EQUIFAX,2nd Quarter (Apr 1 - June 30),True,350000.0
...,...,...,...,...
63,IHS MARKIT,1st Quarter (Jan 1 - Mar 31),False,20000.0
46,ELSEVIER,4th Quarter (Oct 1 - Dec 31),False,10000.0
45,ELSEVIER,3rd Quarter (July 1 - Sep 30),False,30000.0
44,ELSEVIER,2nd Quarter (Apr 1 - June 30),False,20000.0


In [7]:
period_spending['company'] = period_spending['company'].apply(lambda x: x if not x in subsidiaries.keys() else subsidiaries[x])

In [8]:
total_period_spending = period_spending[['company','period','spending']].groupby(['company', 'period']).sum().sort_values(by=['company', 'period']).copy()


In [9]:
ttl = total_period_spending.reset_index().pivot('company', 'period').reset_index()

In [10]:
ttl['2020 Spending'] = ttl.sum(axis=1)

In [11]:
ttl = ttl.sort_values(by='2020 Spending', ascending=False)

Export Spending

In [12]:
ttl.columns = [b if a =="spending" else a for (a, b) in ttl.columns]

In [13]:
ttl.to_csv('../data/findings/period-spending.csv', index=False)

## Issues

In [14]:
d = []
for f in data['Filings']:
    if f['ID'] in df['filing_id'].unique() and f.get('Issues'):
        for issue in f.get('Issues'):
            d.append({
                'filing.id': f['ID'],
                'category': issue['Code'],
                'issue': issue['SpecificIssue']
            })
issues = pd.DataFrame(d)

In [15]:
issues['cleaned'] = issues.issue.str.lower()
issues['hr_bills'] = issues['cleaned'].str.findall('h\.?r\.? ?\d*')
issues['s_bills'] = issues['cleaned'].str.findall('s\. ?\d*')

In [16]:
issues

,filing.id,category,issue,cleaned,hr_bills,s_bills
0,62AEECD8-DF4E-487B-9BDF-987863110E53,COMPUTER INDUSTRY,Monitoring Congressional activities regarding ...,monitoring congressional activities regarding ...,[],[]
1,8636186A-DC3F-40E9-B54D-9F236760075F,CONSUMER ISSUES/SAFETY/PRODUCTS,Issues surrounding data privacy.,issues surrounding data privacy.,[],[]
2,B4D3FEE4-BF23-4A97-A904-6CE4C970B92B,TELECOMMUNICATIONS,General tech issues,general tech issues,[],[]
3,F2965A17-6CA6-4338-97F2-19C4C9188C13,TELECOMMUNICATIONS,"Issues related to robocalling, the Telephone C...","issues related to robocalling, the telephone c...",[hr 946],[]
4,F2965A17-6CA6-4338-97F2-19C4C9188C13,COMPUTER INDUSTRY,"Issues relating to Internet Governance, ICANN,...","issues relating to internet governance, icann,...",[],[s.]
...,...,...,...,...,...,...
957,285C9DF5-8104-4737-8516-0CB0878558B2,HEALTH ISSUES,Issues related to healthcare,issues related to healthcare,[],[]
958,285C9DF5-8104-4737-8516-0CB0878558B2,SCIENCE/TECHNOLOGY,Issues related to technology,issues related to technology,[],[]
959,285C9DF5-8104-4737-8516-0CB0878558B2,FINANCIAL INSTITUTIONS/INVESTMENTS/SECURITIES,Issues related to financial services,issues related to financial services,[],[]
960,43A31771-752F-4F82-97F9-A255BEA23D0D,BUDGET/APPROPRIATIONS,Use of commercial data by federal government a...,use of commercial data by federal government a...,[],[s.]


In [17]:
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    number = "".join(num)
    if len(number) > 0:
        return int(number)
    return np.nan

In [18]:
issues['hr_bills'] = issues['hr_bills'].apply(lambda l: [find_number(x) for x in l])
issues['hr_bills'] = issues['hr_bills'].apply(lambda l: np.nan if len(l) == 0 else l)

issues['s_bills'] = issues['s_bills'].apply(lambda l: [find_number(x) for x in l])
issues['s_bills'] = issues['s_bills'].apply(lambda l: np.nan if len(l) == 0 else l)

issues['hr_bills'] = issues['hr_bills'].apply(lambda l: list(set(l)) if isinstance(l,list) and (len(l) > 0) else np.nan)
issues['s_bills'] = issues['s_bills'].apply(lambda l: list(set(l)) if isinstance(l,list) and (len(l) > 0) else np.nan)

In [19]:
issues['cleaned'] = issues['cleaned'].str.replace('issues related to', '')
issues['cleaned'] = issues['cleaned'].str.replace('issues relating to', '')
issues['cleaned'] = issues['cleaned'].str.replace('issues surrounding', '')

In [20]:
company_issues = issues.merge(df[['filing_id', 'company', 'lobbying_firm', 'period']], left_on='filing.id', right_on='filing_id')
company_issues['company'] = company_issues['company'].apply(lambda x: x if not x in subsidiaries.keys() else subsidiaries[x])

In [21]:
company_issues

,filing.id,category,issue,cleaned,hr_bills,s_bills,filing_id,company,lobbying_firm,period
0,62AEECD8-DF4E-487B-9BDF-987863110E53,COMPUTER INDUSTRY,Monitoring Congressional activities regarding ...,monitoring congressional activities regarding ...,NaN,NaN,62AEECD8-DF4E-487B-9BDF-987863110E53,ORACLE,The Michael Lewan Company,1st Quarter (Jan 1 - Mar 31)
1,8636186A-DC3F-40E9-B54D-9F236760075F,CONSUMER ISSUES/SAFETY/PRODUCTS,Issues surrounding data privacy.,data privacy.,NaN,NaN,8636186A-DC3F-40E9-B54D-9F236760075F,LIVERAMP,The Madison Group,2nd Quarter (Apr 1 - June 30)
2,B4D3FEE4-BF23-4A97-A904-6CE4C970B92B,TELECOMMUNICATIONS,General tech issues,general tech issues,NaN,NaN,B4D3FEE4-BF23-4A97-A904-6CE4C970B92B,ORACLE,Cove Strategies,2nd Quarter (Apr 1 - June 30)
3,F2965A17-6CA6-4338-97F2-19C4C9188C13,TELECOMMUNICATIONS,"Issues related to robocalling, the Telephone C...","robocalling, the telephone consumer protectio...",[946],NaN,F2965A17-6CA6-4338-97F2-19C4C9188C13,NEUSTAR,"Perry Bayliss Government Relations, LLC",2nd Quarter (Apr 1 - June 30)
4,F2965A17-6CA6-4338-97F2-19C4C9188C13,COMPUTER INDUSTRY,"Issues relating to Internet Governance, ICANN,...","internet governance, icann, domain name regis...",NaN,[nan],F2965A17-6CA6-4338-97F2-19C4C9188C13,NEUSTAR,"Perry Bayliss Government Relations, LLC",2nd Quarter (Apr 1 - June 30)
...,...,...,...,...,...,...,...,...,...,...
957,285C9DF5-8104-4737-8516-0CB0878558B2,HEALTH ISSUES,Issues related to healthcare,healthcare,NaN,NaN,285C9DF5-8104-4737-8516-0CB0878558B2,TRANSUNION,Subject Matter (fka Elmendorf Ryan),4th Quarter (Oct 1 - Dec 31)
958,285C9DF5-8104-4737-8516-0CB0878558B2,SCIENCE/TECHNOLOGY,Issues related to technology,technology,NaN,NaN,285C9DF5-8104-4737-8516-0CB0878558B2,TRANSUNION,Subject Matter (fka Elmendorf Ryan),4th Quarter (Oct 1 - Dec 31)
959,285C9DF5-8104-4737-8516-0CB0878558B2,FINANCIAL INSTITUTIONS/INVESTMENTS/SECURITIES,Issues related to financial services,financial services,NaN,NaN,285C9DF5-8104-4737-8516-0CB0878558B2,TRANSUNION,Subject Matter (fka Elmendorf Ryan),4th Quarter (Oct 1 - Dec 31)
960,43A31771-752F-4F82-97F9-A255BEA23D0D,BUDGET/APPROPRIATIONS,Use of commercial data by federal government a...,use of commercial data by federal government a...,NaN,[nan],43A31771-752F-4F82-97F9-A255BEA23D0D,EQUIFAX,"ATRIO CONSULTING GROUP, LLC",4th Quarter (Oct 1 - Dec 31)
